# Global IRF Efficiency Map

Generate global map of CDR efficiency using IRF forcing.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import atlas

## Define experiment and phase



In [3]:
cdr_forcing = "ERW"
vintage = "001"
clobber = False

In [4]:
calc = atlas.global_irf_map(cdr_forcing, vintage)

## Table of experiments

In [5]:
calc.df

,blueprint,polygon,polygon_master,basin,start_date,cdr_forcing,cdr_forcing_file,simulation_key,refdate,stop_n,wallclock,curtail_output
case,,,,,,,,,,,,
smyle.cdr-atlas-v0.control.001,smyle,NaN,NaN,None,1999-01,None,None,baseline,0347-01-01,16,12:00:00,False
smyle.cdr-atlas-v0.glb-erw_North_Atlantic_basin_000_1999-01-01_00000.001,smyle,0.0,0.0,North_Atlantic_basin,1999-01,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_North_Atlantic_basin_000_1999-01-01_00000,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-erw_North_Atlantic_basin_000_1999-04-01_00001.001,smyle,0.0,0.0,North_Atlantic_basin,1999-04,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_North_Atlantic_basin_000_1999-04-01_00001,0347-04-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-erw_North_Atlantic_basin_000_1999-07-01_00002.001,smyle,0.0,0.0,North_Atlantic_basin,1999-07,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_North_Atlantic_basin_000_1999-07-01_00002,0347-07-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-erw_North_Atlantic_basin_000_1999-10-01_00003.001,smyle,0.0,0.0,North_Atlantic_basin,1999-10,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_North_Atlantic_basin_000_1999-10-01_00003,0347-10-01,15,10:00:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...
smyle.cdr-atlas-v0.glb-erw_South_118_1999-10-01_01235.001,smyle,118.0,468.0,South,1999-10,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_South_118_1999-10-01_01235,0347-10-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-erw_South_119_1999-01-01_01236.001,smyle,119.0,469.0,South,1999-01,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_South_119_1999-01-01_01236,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-erw_South_119_1999-04-01_01237.001,smyle,119.0,469.0,South,1999-04,ERW,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-erw_South_119_1999-04-01_01237,0347-04-01,15,10:00:00,True


## Validate the integrations

In [ ]:
calc.df_case_status

In [ ]:
calc.clobber_case("smyle.cdr-atlas-v0.glb-erw_North_Atlantic_basin_087_1999-07-01_00350.001")

In [ ]:
%%time
calc.validate(clobber=clobber)

In [ ]:
df = calc.df_validation
df

In [ ]:
df.ALK_ALT_CO2_rmse.plot.hist(bins=20)

In [ ]:
df.DIC_ALT_CO2_rmse.plot.hist(bins=20)

In [ ]:
df.FG_ALT_CO2_rmse.plot.hist(bins=20)

## Analysis

In [ ]:
calc.analyze(clobber=False)

In [ ]:
calc.df_analysis

In [ ]:
import warnings
warnings.filterwarnings("error")

fig, ax = plt.subplots(1, 1, figsize=(6, 4))

for row_data in tqdm(calc.df_analysis.iterrows()):
    case = row_data[0]
    path_zarr = row_data[1].iloc[0]   
    try:
        ds = xr.open_zarr(path_zarr, decode_timedelta=False)
        da = ds["DIC_ADD_TOTAL"]
        if ds.time.values[0].year > 500:
            print(f"bad time:\n\t{case}\n\t{path_zarr}")        
    except:
        print(f"FAILED\n\t{case}\n\t{path_zarr}")
        raise
    da.plot(ax=ax, label=case)

## Visualization

In [ ]:
#calc.visualize(clobber=True)

In [ ]:
df = calc.df_case_status
df